In [ ]:
import glob
import os
import yaml

import numpy as np
import pandas as pd
import pickle
import seaborn as sns
from matplotlib import pyplot as plt

from hera_cal.io import HERAData

from align_utils import idr2_jdsx
from fit_diagnostics import abs_residuals, norm_residuals
from plot_utils import clipped_heatmap, df_heatmap, flagged_hist, \
plot_res_grouped, plot_res_heatmap
from red_likelihood import group_data, makeCArray, relabelAnts, split_rel_results
from red_utils import find_nearest, find_zen_file
from xd_utils import union_bad_ants, XDgroup_data

In [ ]:
%matplotlib inline

In [ ]:
plot_figs = False
if plot_figs:
    import matplotlib as mpl
    mpl.rcParams['figure.dpi'] = 300

In [ ]:
jd_time = 2458098.43869 # used to find LST that labels dataframe
pol = 'ee'
ndist = 'gaussian'
dir_path = 'xd_rel_dfs'

In [ ]:
lst_df = pd.read_pickle('jd_lst_map_idr2.pkl')

In [ ]:
lst_ref = lst_df[lst_df['JD_time'] == jd_time]['LASTs'].values[0][0]
xd_df_path = os.path.join(dir_path, 'xd_rel_df.{:.4f}.{}.{}.pkl'.format(lst_ref, pol, ndist))

In [ ]:
with open(os.path.join(dir_path, 'xd_rel_df.{:.4f}.{}.md.pkl'.format(lst_ref, pol)), 'rb') as f:
    md = pickle.load(f)
    
xd_df = pd.read_pickle(xd_df_path)

chans = xd_df.index.get_level_values(level='freq').unique().values
tints = xd_df.index.get_level_values(level='time_int').unique().values

Nfreqs = chans.size
Ntints = tints.size

xd_df.sample(5).sort_index()

## Performance

### Number of iterations

In [ ]:
plot_res_grouped(xd_df, 'nit', logy=True)

In [ ]:
plot_res_heatmap(xd_df, 'nit', clip=True)

### Log-likelihood

In [ ]:
plot_res_grouped(xd_df, 'fun', logy=True, figsize=(10, 7))

In [ ]:
plot_res_heatmap(xd_df, 'fun', clip=True, clip_pctile=98, figsize=(8, 6))

### Residuals

In [ ]:
xd_df[['med_abs_norm_res_Re', 'med_abs_norm_res_Im']] = xd_df.apply(lambda row: \
    pd.Series(abs_residuals(row['norm_residual'])), axis=1)

xd_df['med_abs_norm_res_comb'] = np.sqrt(xd_df['med_abs_norm_res_Re']**2 + \
                                         xd_df['med_abs_norm_res_Im']**2)

In [ ]:
plot_res_heatmap(xd_df, 'med_abs_norm_res_comb', vmin=0.16, vmax=0.22, \
                 figsize=(8, 6))

## Gains at sample frequency and time slice

In [ ]:
# Check results for a given frequency & time integration
test_freq = 600
test_tint = 53

resx = xd_df.loc[(test_freq, test_tint)][5:-5].values.astype(float)
test_vis, test_gains = split_rel_results(resx, md['no_unq_bls'], coords='cartesian')
test_gains = test_gains.reshape((md['JDs'].size, -1))

print('Mean gain amplitude across JDs for test frequency {} and time integration {}: '\
      '\n{}\n'.format(test_freq, test_tint, np.mean(np.abs(test_gains), axis=0)))
print('Mean gain phase across JDs for test frequency {} and time integration {}: '\
      '\n{}\n'.format(test_freq, test_tint, np.mean(np.angle(test_gains), axis=0)))

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(9, 3.5), sharey=True)

sns.heatmap(np.abs(test_gains), cmap=sns.cm.rocket_r, center=1, ax=ax1)
sns.heatmap(np.angle(test_gains), cmap='bwr', center=0, ax=ax2, vmin=-np.pi, vmax=np.pi)
ax1.set_xlabel('Antenna number')
ax2.set_xlabel('Antenna number')

ax1.set_yticks(np.arange(md['JDs'].size)+0.5)
ax1.set_yticklabels(md['JDs'], rotation=0)
ax2.set_yticks(np.arange(md['JDs'].size), minor=True)
ax1.tick_params(axis='y', which='minor', color='white')
ax2.tick_params(axis='y', which='minor', color='white')

ax1.set_xticks(np.arange(md['no_ants'])[::5]+0.5, minor=False)
ax1.set_xticklabels(np.arange(md['no_ants'])[::5])
ax2.set_xticks(np.arange(md['no_ants'])[::5]+0.5, minor=False)
ax2.set_xticklabels(np.arange(md['no_ants'])[::5])

ax1.grid(which='minor', axis='y', linestyle='--', lw=0.5)
ax2.grid(which='minor', axis='y', linestyle='--', lw=0.5)

plt.tight_layout()
plt.show()

## Visibilities

In [ ]:
no_unq_bls = md['no_unq_bls']
no_min_p = 5 # number of columns in df that are attributes of the SciPy OptimizeResult 
vis_df = xd_df.iloc[:, no_min_p:no_unq_bls*2+no_min_p]

visC_df = vis_df.apply(lambda row: makeCArray(row.values), axis=1)
visC_df = pd.DataFrame(visC_df.values.tolist(), index=visC_df.index)

### Visibilities at test time integration

In [ ]:
df = visC_df.xs(53, level='time_int').abs().transpose()
vmax = np.nanpercentile(df.values, 98)
vmin = np.nanpercentile(df.values, 2)
df_heatmap(df, xbase=25, ybase=5, \
           xlabel='Channel', ylabel='Redundant Baseline Group', \
           vmin=vmin, vmax=vmax, figsize=(8, 6))

In [ ]:
df = visC_df.xs(53, level='time_int').applymap(np.angle).transpose()
df_heatmap(df, xbase=25, ybase=5, cmap='bwr', vmin=-np.pi, vmax=np.pi, center=0, \
           xlabel='Channel', ylabel='Redundant Baseline Group', figsize=(8, 6))

## Flagging methods

#### Get flags from .smooth_abs.calfits files

In [ ]:
idr2_flags = np.load('idr2_flags.npz')['arr_0'].astype(bool)
idr2_flags = idr2_flags[:, chans - 50, :]

#### MAD-clipping flags from LST-Binning

In [ ]:
if os.path.exists('/lustre/aoc/projects/hera/H1C_IDR2/'):
    local_work = False
    lst_binned_dir = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/LSTBIN/one_group/grp1'
else:
    lst_binned_dir = '/Users/matyasmolnar/Downloads/HERA_Data/robust_cal/'
    local_work = True
    
lst_binned_file = os.path.join(lst_binned_dir, 'zen.grp1.of1.LST.1.40949.HH.OCRSL.uvh5')

In [ ]:
hd_lstb, _, LSTdata = group_data(lst_binned_file, pol, chans=chans, tints=None, \
                                 bad_ants=union_bad_ants(idr2_jdsx), \
                                 flag_path=None, noise=False)

In [ ]:
hd_lstb.lsts

In [ ]:
hd = HERAData(find_zen_file(jd_time))
hd.lsts

In [ ]:
lst_binned_files = glob.glob(os.path.join(lst_binned_dir, 'zen.grp1.of1.LST.*.HH.OCRSL.uvh5'))
dtst_lsts = [float('.'.join(os.path.basename(lst_f).split('.')[4:6])) for lst_f in lst_binned_files]

In [ ]:
lst_idx1 = find_nearest(dtst_lsts, hd.lsts[0], condition='leq')[1]
lst_idx2 = find_nearest(dtst_lsts, hd.lsts[0], condition='geq')[1]

In [ ]:
hd_lstb1, _, LSTdata1 = group_data(lst_binned_files[lst_idx1], pol, chans=chans, 
                                   tints=None, bad_ants=union_bad_ants(idr2_jdsx), \
                                   flag_path=None, noise=False)

hd_lstb2, _, LSTdata2 = group_data(lst_binned_files[lst_idx2], pol, chans=chans, 
                                   tints=None, bad_ants=union_bad_ants(idr2_jdsx), \
                                   flag_path=None, noise=False)

concat_lsts = np.concatenate((hd_lstb1.lsts, hd_lstb2.lsts))
if (hd.lsts[0] < concat_lsts).all() or (hd.lsts[-1] > concat_lsts).all():
    raise Exception('Could not find LST-binned files.')

In [ ]:
align_idxs = []
for i in hd.lsts:
    align_idxs.append(find_nearest(concat_lsts, i)[1])

In [ ]:
hd.lsts

In [ ]:
concat_lsts[align_idxs]

In [ ]:
# nsamples[(12, 13, 'ee')]

In [ ]:
# data[(12, 13, 'ee')]

#### Additional flags by Nick Kern

In [ ]:
if local_work:
    nkern_flg_dir = lst_binned_dir
else:
    nkern_flg_dir = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2_pspec/v2/one_group/'

nkern_flg_file = os.path.join(nkern_flg_dir, 'preprocess_params.yaml')

# Read YAML file
with open(nkern_flg_file, 'r') as stream:
    data_loaded = yaml.safe_load(stream)

In [ ]:
man_flags = np.concatenate([np.arange(i[0], i[1]+1) for i in \
                            data_loaded['algorithm']['fg_filt']['flag_chans']]).ravel()
rel_nflags = man_flags[np.where(np.logical_and(man_flags >= chans[0], man_flags <= chans[-1]))] -  chans[0]
idr2_flags[:, rel_nflags, :] = True

In [ ]:
# TODO
# Get mad-clipped flags at LST-binning stage

#### Flags used in H1C_IDR2 limits paper

In [ ]:
fig, ax = clipped_heatmap(idr2_flags.sum(axis=0).transpose(), 'Time Integration', 'Channel', \
                          vmin=0, clip_pctile=100, figsize=(8, 5), xoffset=-chans[0])
plt.tight_layout()
plt.show()

### Negative log-likelihood histograms

In [ ]:
nll_upper_cut = 1
xd_rel_values = xd_df['fun'].values.astype(float)

flgs_all = np.zeros_like(idr2_flags[0, ...])
flg_pct = 50 / 100
flgs_all[np.where(idr2_flags.mean(axis=0) > flg_pct)] = True # if 50% of days flagged
flgs_all = flgs_all.ravel(order='F')

flagged_hist(xd_rel_values, flgs_all, \
             xlabel=r'$-\ln(\mathcal{L}^C_\mathrm{xd\_rel})$', \
             lower_cut=0.1, upper_cut=nll_upper_cut, bin_width=0.02, hist_start=0, ylim=(0, 1500))

In [ ]:
sus_slices = np.where((xd_rel_values > 10) & ~flgs_all)[0]
xd_df.index[sus_slices]

In [ ]:
xd_rel_values[sus_slices]

### Calculating the NLLs had the minimization been done with Gaussian distribution

In [ ]:
if ndist == 'gaussian':
    nll_dist = 'cauchy'
if ndist == 'cauchy':
    nll_dist = 'gaussian'

In [ ]:
res_cols = [col for col in xd_df.columns.values if col.isdigit()]

# Retrieve solved gains in array format
xd_gains = xd_df[res_cols[no_unq_bls*2:]].values.reshape((Nfreqs, Ntints, md['JDs'].size, -1))
xd_gains = np.moveaxis(xd_gains, [2, 0, 1, 3], [0, 1, 2, 3])
y = xd_gains.reshape(xd_gains.shape[:3] + (md['no_ants'], -1, 2))
xd_gains = np.squeeze(y[..., 0] + 1j*y[..., 1])

# Retrieve solved visibilities in array format
xd_vis = xd_df[res_cols[:no_unq_bls*2]].values.reshape((Nfreqs, Ntints, -1, 2))
xd_vis = xd_vis[..., 0] + 1j*xd_vis[..., 1]
xd_vis = np.tile(np.expand_dims(xd_vis, axis=0), (md['JDs'].size, 1, 1, 1))

In [ ]:
if Ntints == md['Ntimes']:
    tints = None
if (md['JDs'] == idr2_jdsx).all():
    jds = idr2_jdsx
else:
    jds = md['JDs']

if os.path.exists('test_idr2_cdata.npz'):
    local_work = True
    local_jd = 2458098.43869
    if jd_time == local_jd:
    # retrieve data locally
        cdata = np.load('test_idr2_cdata.npz')['arr_0']
        cndata = np.load('test_idr2_cndata.npz')['arr_0']
    else:
        raise Exception('Only H1C_IDR2 visibility data across JDs aligned with {} '
                        'is available locally.'.format(local_jd))
else:
    _, _, cdata, cndata = XDgroup_data(jd_time, jds, pol, chans=chans, \
        tints=tints, bad_ants=True, use_flags='first', noise=True)
    cdata = cdata.data

credg = relabelAnts(md['redg'])     

In [ ]:
NLLFN = {'cauchy': lambda delta: np.log(1 + np.square(np.abs(delta))).sum(axis=(0, -1)),
         'gaussian': lambda delta: np.square(np.abs(delta)).sum(axis=(0, -1))}

In [ ]:
credg = relabelAnts(md['redg'])
gvis = xd_vis[..., credg[:, 0]]*xd_gains[..., credg[:, 1]]*np.conj(xd_gains[..., credg[:, 2]])
delta = cdata - gvis
nlog_likelihood = NLLFN[nll_dist](delta)

In [ ]:
fig, ax = clipped_heatmap(nlog_likelihood.transpose(), ylabel='Time integration', 
                          clip_pctile=98, figsize=(8, 6), clip_rnd=100, xoffset=-chans[0])
plt.tight_layout()
plt.show()

### Residuals between solved visibilities and gain transformed observed visibilities

With across days redundant calibration, we obtain a single set of visibilities. We wish to compare these solved visibilities to the observed visibilities on different days, to find potential outliers. The amplitudes of these visibilities could be compared; their phases, however, cannot, since there are degenerate offsets between them. We do not wish to calculate these degenerate offsets, as this is computationally expensive - this would require doing pairs of comparison between the solved xd_rel_cal solutions and each day. What we can do, however, is take the observed visibilities and divide them by the solved gains to get a quantity that is comparable as it is degenerately consistent. The residual between this quantity and the true visibilities is what we use for outlier detection.

In [ ]:
tr_vis = cdata / xd_gains[..., credg[:, 1]] / np.conj(xd_gains[..., credg[:, 2]])
tr_res = xd_vis[..., credg[:, 0]] - tr_vis

#### Modified Z-score

In [ ]:
correction = 1.4826
mad = np.median(np.abs(tr_res), axis=0) # Median Absolute Deviation
modz = np.abs(tr_res)/(correction*np.tile(np.expand_dims(mad, axis=0), \
                                          (md['JDs'].size, 1, 1, 1))) # Modified Z-score
# Note that these quantities are about the solve visibility values, and not
# about their medians

In [ ]:
mean_modz = np.mean(modz, axis=(0, -1)) # mean over days and baselines
fig, ax = clipped_heatmap(mean_modz.transpose(), 'Time Integration', 'Channel', \
                          clip_pctile=99, figsize=(8, 5), xoffset=-chans[0])
plt.tight_layout()
plt.show()

In [ ]:
# mean modified Z-score
bad_slicesz = np.where(np.logical_and(modz.mean(axis=-1) > 0.8, ~idr2_flags))
print('{} potentially bad slices found that are not flagged through the '\
      'hera_cal pipeline, through z-score considerations'.format(bad_slicesz[0].size))

In [ ]:
# look at individual baselines
bad_slicesz_bl = np.where(np.logical_and(modz > 5.5, \
    ~np.tile(np.expand_dims(idr2_flags, axis=-1), (1, 1, 1, md['redg'].shape[0]))))

In [ ]:
bad_slicesz_bl

#### $\mathcal{R}_{\mathrm{man}}$

$\mathcal{R}_{\mathrm{man}}$ already calculated above, but run on last baseline dimension too - should be similar results to modified Z-score

In [ ]:
nrm_resid = norm_residuals(xd_vis[..., credg[:, 0]], tr_vis)
abs_resid = np.median(np.abs(nrm_resid), axis=-1) # median over the baseline axis

In [ ]:
mean_abs_resid = np.mean(abs_resid, axis=0) # mean over days
vmin = np.nanpercentile(mean_abs_resid, 1)
fig, ax = clipped_heatmap(mean_abs_resid.transpose(), 'Time Integration', 'Channel', \
                          clip_pctile=98, vmin=vmin, figsize=(8, 5), xoffset=-chans[0])
plt.tight_layout()
plt.show()

In [ ]:
idr2_jdsx

In [ ]:
# TODO
# Histograms of NLL, NLL / Noise, R_man to find outliers

#### LST-Binned files that contain MAD Clipping